In [1]:
import pandas as pd
from nltk import tokenize, stem
from nltk.corpus import stopwords
from re import sub
from unidecode import unidecode
from multiprocessing import Pool
import math
import string

In [23]:
def tokenizer(tweets):
    
    """
    Arguments:
    
    tweets - List of strings, the strings being tweets.
    -----------------------------------------------------------------
    Return-value:
    
    tokens - 2-dimensional list containing tokens as strings.
    
    """
    
    tokens = list()
    tk = tokenize.TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)
    for tweet in tweets:
        try:
            element = tk.tokenize(tweet)
        except UnicodeDecodeError:
            element = []
        tokens.append(element)
    return tokens


def normalize(tokens):
    
    """
    Arguments:
    
    tokens - List of lists containing tokens of the tweets.
    -----------------------------------------------------------------
    Return-value:
    
    final_tokens - List of lists containing normalized tokens of the tweets.
    
    """
    
    stop = stopwords.words('english')
    exclude = set(string.punctuation)
    stemmer = stem.PorterStemmer()
    
    # Unicode to string
    tokens_str = [[unidecode(token) for token in tweet] for tweet in tokens]
    
    #Replacing URLs with empty string
    tokens_str = [[sub(r"http\S+", "", token) for token in tweet] for tweet in tokens_str]
    
    # Removing punctuation    
    tokens_str = [[''.join(ch for ch in token if ch not in exclude) for token in tweet] for tweet in tokens_str]
    
    # Removing numbers
    tokens_str = [[sub(r'\d+', '', token) for token in tweet] for tweet in tokens_str]
    
    # Stemming
    tokens_str = [[stemmer.stem(token) for token in tweet] for tweet in tokens_str]
    
    # Unicode to string
    tokens_str = [[unidecode(token) for token in tweet] for tweet in tokens_str]
    
    # Removing stopwords
    tokens_str = [[word for word in tweet if word not in stop] for tweet in tokens_str]
    
    # Removing empty tokens (strings)
    final_tokens = list()
    for str_list in tokens_str:
        x = filter(None, str_list)
        final_tokens.append(x)
        
    # Removing periods in abbreviations. Ex: U.S.A. to USA

    # for tweet in tokens:
    #     tweet = [sub(r'(?<!\w)([A-Z])\.', r'\1', x.lower()) for x in tweet]
    #     print (tweet)
    
    return final_tokens


def createInvertedIndex(tokens):
    
    """
    Arguments:
    
    tokens - List of lists containing normalized tokens.
    ------------------------------------------------------------------
    Return-value:
    
    inverted_index - Dictionary containing the index words.
    
    """
    
    inverted_index = {}
    for i in range(len(tokens)):
        for j in range(len(tokens[i])):
            if tokens[i][j]:
                if inverted_index.has_key(tokens[i][j]):
                    if i not in inverted_index[tokens[i][j]]:
                        inverted_index[tokens[i][j]].append(i)
                else:
                    inverted_index[tokens[i][j]] = [i]
#         print(i)
    return inverted_index


def bm25(query_tokens, index, tokens):
    
    map_id_score = {}
    k1 = 1.2
    b = 0.75 
    N = len(tokens) # Number of documents in the collection
    adder = 0
    
    for doc_tokens in tokens:
        adder += len(doc_tokens)
    
    avg_doclen = float(adder)/N
    
    for j in range(len(tokens)):
        
        lend = float(len(tokens[j]))
        score = 0
        
        for i in range(len(query_tokens)):
            
            if query_tokens[i] in tokens[j]:
                
                n = float(len(index[query_tokens[i]]))
                f = float(tokens[j].count(query_tokens[i]))
                T1 = math.log(float(N-n+0.5)/(n+0.5),2)
                x = k1 * ((1-b) + b*(lend/avg_doclen)) + f
                T2 = float((k1+1)*f)/x
                score += T1*T2
        
        map_id_score[j] = score
        
    return map_id_score

def generateFeatures(i, sorted_scores, threshold):
    
    """
    Arguments:
    
    i - ID of the tweet in the test dataset
    sorted_scores - List of tuples containing TweetID and Similarity score, sorted in descending order of scores
    threshold - Number of results to retrieve from the ranking generated.
    -------------------------------------------------------------------------------------------------------------
    Return-value:
    
    features - Dictionary containing 24 features
    
    """
    global map_id_sentiment
    
    # Features dictionary
    features = {
        'avg_n' : 0.0,
        'max_n' : 0.0,
        'min_n' : 0.0,
        'sum_n' : 0.0,
        'count_n' : 0.0,
        'phi_n' : 0.0,
        'phi_pos_n' : 0.0,
        'phi_avg_n' : 0.0,
        'phi_max_n' : 0.0,
        'phi_min_n' : 0.0,
        'phi_sum_n' : 0.0,
        'phi_count_n' : 0.0,
        'avg_p' : 0.0,
        'max_p' : 0.0,
        'min_p' : 0.0,
        'sum_p' : 0.0,
        'count_p' : 0.0,
        'phi_p' : 0.0,
        'phi_pos_p' : 0.0,
        'phi_avg_p' : 0.0,
        'phi_max_p' : 0.0,
        'phi_min_p' : 0.0,
        'phi_sum_p' : 0.0,
        'phi_count_p' : 0.0 
    }
    
    # Calculating aggregation functions feature values
    for i in range(0, threshold):
        if map_id_sentiment[sorted_scores[i][0]] == 0:
            features['count_n'] += 1
            features['sum_n'] += sorted_scores[i][1]
            if features['count_n'] == 1.0:
                features['max_n'] = sorted_scores[i][1]
                features['min_n'] = sorted_scores[i][1]
            elif sorted_scores[i][1] < features['min_n']:
                features['min_n'] = sorted_scores[i][1]
        else:
            features['count_p'] += 1
            features['sum_p'] += sorted_scores[i][1]
            if features['count_p'] == 1.0:
                features['max_p'] = sorted_scores[i][1]
                features['min_p'] = sorted_scores[i][1]
            elif sorted_scores[i][1] < features['min_p']:
                features['min_p'] = sorted_scores[i][1]
    
    if features['count_n'] > 0.0:
        features['avg_n'] = features['sum_n']/features['count_n']
    if features['count_p'] > 0.0:
        features['avg_p'] = features['sum_p']/features['count_p']
    
    # Calculating phi feature values
    
    rank_rel_n = 0
    rank_rel_p = 0
    
    for i in range(0, threshold):
        if map_id_sentiment[sorted_scores[i][0]] == 0:
            rank_rel_n += 1
            features['phi_n'] += float(rank_rel_n)/(i+1)
            features['phi_pos_n'] += (float(rank_rel_n)/(i+1)) * sorted_scores[i][1]
        else:
            rank_rel_p += 1
            features['phi_p'] += float(rank_rel_p)/(i+1)
            features['phi_pos_p'] += (float(rank_rel_p)/(i+1)) * sorted_scores[i][1]
    
    if features['avg_n'] > 0.0:
        features['phi_avg_n'] = features['phi_n']/features['avg_n']
    if features['max_n'] > 0.0:
        features['phi_max_n'] = features['phi_n']/features['max_n']
    if features['min_n'] > 0.0:
        features['phi_min_n'] = features['phi_n']/features['min_n']
    if features['count_n'] > 0.0:
        features['phi_count_n'] = features['phi_n']/features['count_n']
    if features['sum_n'] > 0.0:
        features['phi_sum_n'] = features['phi_n']/features['sum_n']
    
    if features['avg_p'] > 0.0:
        features['phi_avg_p'] = features['phi_p']/features['avg_p']
    if features['max_p'] > 0.0:
        features['phi_max_p'] = features['phi_p']/features['max_p']
    if features['min_p'] > 0.0:
        features['phi_min_p'] = features['phi_p']/features['min_p']
    if features['count_p'] > 0.0:
        features['phi_count_p'] = features['phi_p']/features['count_p']
    if features['sum_p'] > 0.0:
        features['phi_sum_p'] = features['phi_p']/features['sum_p']
    
    return features
    
    

In [22]:
print("Reading training and testing dataset...")

df_full = pd.read_csv('stanford/training.1600000.processed.noemoticon.csv', header=None)
df_testdata = pd.read_csv('stanford/testdata.manual.2009.06.14.csv', header=None)
df_test = df_testdata.loc[df_testdata[0].isin([0,4])]
df_test.reset_index(inplace=True, drop=True)

print("Done.")

Reading training and testing dataset...
Done.


In [4]:
df1 = df_full.iloc[:30000]
df2 = df_full.iloc[800000:830000]
df = pd.concat([df1,df2])
tweets = list(df[5])
df.reset_index(inplace=True, drop=True)

In [5]:
print("Mapping tweetIDs to tweet and sentiment...")

map_id_tweet = {}
map_id_sentiment = {}

for i in range(len(df)):
    map_id_tweet[i] = df.iloc[i][5]
    map_id_sentiment[i] = df.iloc[i][0]

# for i in range(len(df2)):
#     map_id_tweet[i] = df2.iloc[i][5]
#     map_id_sentiment[i] = df2.iloc[i][0]

print("Done.")
# pool = Pool()
# tokens = pool.map(tokenizer, tweets)

Mapping tweetIDs to tweet and sentiment...
Done.


In [6]:
# Tokenizing the tweeets
print("Tokenizing tweets...")
tokenized_tokens = tokenizer(tweets)
print("Done.")

print(tokenized_tokens[:3])

Tokenizing tweets...
Done.
[[u'http://twitpic.com/2y1zl', u'-', u'awww', u',', u"that's", u'a', u'bummer', u'.', u'you', u'shoulda', u'got', u'david', u'carr', u'of', u'third', u'day', u'to', u'do', u'it', u'.', u';D'], [u'is', u'upset', u'that', u'he', u"can't", u'update', u'his', u'facebook', u'by', u'texting', u'it', u'...', u'and', u'might', u'cry', u'as', u'a', u'result', u'school', u'today', u'also', u'.', u'blah', u'!'], [u'i', u'dived', u'many', u'times', u'for', u'the', u'ball', u'.', u'managed', u'to', u'save', u'50', u'%', u'the', u'rest', u'go', u'out', u'of', u'bounds']]


In [7]:
# Normalizing the tweets
print("Normalizing tweets...")
normalized_tokens = normalize(tokenized_tokens)
print("Done.")
print(normalized_tokens[:3])

Normalizing tweets...
Done.
[['awww', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'D'], ['upset', 'cant', 'updat', 'hi', 'facebook', 'text', 'might', 'cri', 'result', 'school', 'today', 'also', 'blah'], ['dive', 'mani', 'time', 'ball', 'manag', 'save', 'rest', 'go', 'bound']]


In [8]:
# Creating Inverted Index
print("Creating Inverted Index...")
index = createInvertedIndex(normalized_tokens)
print("Done.")


Creating Inverted Index...
Done.


In [24]:
print("Preprocessing test queries...")
queries = list(df_test[5])
tokens1 = tokenizer(queries)
query_tokens = normalize(tokens1)
# print(query_tokens)
print("Done.")

Preprocessing test queries...
Done.


In [25]:
print("\nNumber of tweets in the corpus: " + str(len(tweets)))
print("Number of index terms: " + str(len(index)))
threshold = 25
features_list = list()
print("\nPrinting the top 10 results for each query:\n\n")

for i in range(0,len(queries)):
    map_id_score = bm25(query_tokens[i], index, normalized_tokens)
    data = map_id_score.items()
    sorted_scores = sorted(data, key=lambda x: x[1],reverse=True)
    query_features = generateFeatures(i, sorted_scores, threshold)
    features_list.append(query_features)
    print(i)
#     print("\n\nQuery : " + str(queries[i]))
#     print("\nMost Relevant Results : \n")
#     for j in range(threshold):
#         print("[" + str(j+1) + "] : (Score = " + str(round(sorted_scores[j][1],4)) + ") " + df.iloc[sorted_scores[j][0]][5])

features_df = pd.DataFrame(features_list)
# print(features_df)
features_df.to_csv('features.csv')


Number of tweets in the corpus: 60000
Number of index terms: 29624

Printing the top 10 results for each query:


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248

In [26]:
print("Hello")

Hello
